In [ ]:
import requests
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
import backoff
from requests.exceptions import RequestException
import pandas as pd
import numpy as np
import mysql.connector
from sqlalchemy import create_engine
import threading
import math
import pickle
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, FloatType

In [ ]:
def connect_db(config):
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    return cnx, cursor

config_gsql = {
    'user': 'eth_project',
    'password': 'eth_000!',
    'host': 'localhost',
    'port': '3306',
    'database': 'ETHProjectDB'
}

In [ ]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("process_df_tx") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "32g") \
    .getOrCreate()

In [ ]:
url = f"jdbc:mariadb://{config_gsql['host']}:{config_gsql['port']}/{config_gsql['database']}"
user = config_gsql['user']
password = config_gsql['password']
block_df = spark.read.format('jdbc').options(url=url, dbtable='Blocks', user=user, password=password).load()
address_df = spark.read.format('jdbc').options(url=url, dbtable='Addresses', user=user, password=password).load()
category_df = spark.read.format('jdbc').options(url=url, dbtable='TxCategories', user=user, password=password).load()